## Predict Crypto-Price



#### Load data

In [1]:
#pip install python-binance  ta

In [2]:
import pandas as pd
import ta
import requests


In [3]:
#API = 2fd7d110c2283b5ebc80bc2b33cf63226b5e5cefd82696595f6bde8063fca87f


api_key = '2fd7d110c2283b5ebc80bc2b33cf63226b5e5cefd82696595f6bde8063fca87f'  

url = "https://min-api.cryptocompare.com/data/v2/histohour"   

parameters = {
    'fsym': 'BTC',       # from symbol
    'tsym': 'USD',       # to symbol
    'limit': 2000,        # number of days
    'api_key': api_key
}

response = requests.get(url, params=parameters)
data = response.json()

# Check if the response is OK
if data['Response'] == 'Success':
    ohlcv = data['Data']['Data']
    df = pd.DataFrame(ohlcv)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    
    print(df.head())
else:
    print("Error fetching data:", data['Message'])


                         high       low      open  volumefrom      volumeto  \
time                                                                          
2025-02-26 14:00:00  87618.79  85397.67  87105.93     6540.77  5.653567e+08   
2025-02-26 15:00:00  88443.93  87322.84  87532.93     3789.81  3.328138e+08   
2025-02-26 16:00:00  87707.96  86196.02  87462.81     3352.34  2.912486e+08   
2025-02-26 17:00:00  87203.05  85606.44  86715.99     2684.13  2.322850e+08   
2025-02-26 18:00:00  86410.64  83478.28  85964.60     9906.15  8.392586e+08   

                        close conversionType conversionSymbol  
time                                                           
2025-02-26 14:00:00  87532.93         direct                   
2025-02-26 15:00:00  87462.81         direct                   
2025-02-26 16:00:00  86715.99         direct                   
2025-02-26 17:00:00  85964.60         direct                   
2025-02-26 18:00:00  84203.28         direct                  

In [4]:
data.keys()

dict_keys(['Response', 'Message', 'HasWarning', 'Type', 'RateLimit', 'Data'])

In [5]:
data['Response']

'Success'

#### Prepare dataframe and add indicators

In [6]:
# 1. Add technical indicators

df['MA20'] = df['close'].rolling(window=20).mean()
df['RSI'] = ta.momentum.RSIIndicator(df['close']).rsi()
df['MACD'] = ta.trend.MACD(df['close']).macd()
df['MACD_signal'] = ta.trend.MACD(df['close']).macd_signal()


In [7]:
df.head(2)

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,MA20,RSI,MACD,MACD_signal
time,,,,,,,,,,,,
2025-02-26 14:00:00,87618.79,85397.67,87105.93,6540.77,5.653567e+08,87532.93,direct,,NaN,NaN,NaN,NaN
2025-02-26 15:00:00,88443.93,87322.84,87532.93,3789.81,3.328138e+08,87462.81,direct,,NaN,NaN,NaN,NaN


In [8]:
# Lag features (price from previous hour)
df['Lag_1'] = df['close'].shift(1)
df['Lag_2'] = df['close'].shift(2)

In [9]:
df.head(2)

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2
time,,,,,,,,,,,,,,
2025-02-26 14:00:00,87618.79,85397.67,87105.93,6540.77,5.653567e+08,87532.93,direct,,NaN,NaN,NaN,NaN,NaN,NaN
2025-02-26 15:00:00,88443.93,87322.84,87532.93,3789.81,3.328138e+08,87462.81,direct,,NaN,NaN,NaN,NaN,87532.93,NaN


## Preprocess data

Handle null values

In [10]:
df.isna().sum()

high                 0
low                  0
open                 0
volumefrom           0
volumeto             0
close                0
conversionType       0
conversionSymbol     0
MA20                19
RSI                 13
MACD                25
MACD_signal         33
Lag_1                1
Lag_2                2
dtype: int64

In [11]:
#Drop null values (first 26 rows of the data)
df.dropna(inplace = True)

#### Save dataframe locally

In [12]:
df.to_csv("crypto_hourly_data.csv", index=False)

## Feature Engineering

In [13]:
df.drop('conversionSymbol', axis = 1, inplace = True)

#### Handle categorical data

In [14]:
df['conversionType'].nunique()

1

In [15]:
df['conversionType'] = df['conversionType'].map({'direct' : 1}).astype('i1')

In [16]:
df.head(2)

,high,low,open,volumefrom,volumeto,close,conversionType,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2
time,,,,,,,,,,,,,
2025-02-27 23:00:00,84858.26,84337.62,84636.47,1102.55,93244758.94,84657.34,1,85309.2030,44.921644,-453.892146,-384.089907,84636.47,84299.28
2025-02-28 00:00:00,84822.58,84159.05,84657.34,984.01,83145894.23,84200.67,1,85264.9565,41.352537,-461.131373,-399.498200,84657.34,84636.47


#### Handle Datetime feature

In [17]:
#extract datetime features
df['day'] = df.index.day
df['year'] = df.index.year
df['month'] = df.index.month
df['day_name'] = df.index.day_name()

In [18]:
df.head()

,high,low,open,volumefrom,volumeto,close,conversionType,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2,day,year,month,day_name
time,,,,,,,,,,,,,,,,,
2025-02-27 23:00:00,84858.26,84337.62,84636.47,1102.55,9.324476e+07,84657.34,1,85309.2030,44.921644,-453.892146,-384.089907,84636.47,84299.28,27,2025,2,Thursday
2025-02-28 00:00:00,84822.58,84159.05,84657.34,984.01,8.314589e+07,84200.67,1,85264.9565,41.352537,-461.131373,-399.498200,84657.34,84636.47,28,2025,2,Friday
2025-02-28 01:00:00,84330.96,81131.57,84200.67,6654.60,5.480585e+08,81614.91,1,85054.5130,27.856612,-667.819526,-453.162465,84200.67,84657.34,28,2025,2,Friday
2025-02-28 02:00:00,82815.88,79532.20,81614.91,9936.50,8.026110e+08,80759.62,1,84782.1905,24.955427,-890.372472,-540.604467,81614.91,84200.67,28,2025,2,Friday
2025-02-28 03:00:00,81495.86,80192.01,80759.62,4598.99,3.718406e+08,80375.99,1,84489.6555,23.760120,-1085.193584,-649.522290,80759.62,81614.91,28,2025,2,Friday


#### Handle categorical datetime features

In [19]:
#import encoder library
from sklearn.preprocessing import LabelEncoder

In [20]:
encoder = LabelEncoder()   #intialize encoder
df['day_name'] = encoder.fit_transform(df['day_name'])
df['day'] = encoder.fit_transform(df['day'])
df['month'] =  encoder.fit_transform(df['month'])

In [21]:
df['year'].unique()

array([2025])

In [22]:
df['year'] = df['year'].map({2025 : 1}).astype('i1')

In [23]:
df.head(5)

,high,low,open,volumefrom,volumeto,close,conversionType,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2,day,year,month,day_name
time,,,,,,,,,,,,,,,,,
2025-02-27 23:00:00,84858.26,84337.62,84636.47,1102.55,9.324476e+07,84657.34,1,85309.2030,44.921644,-453.892146,-384.089907,84636.47,84299.28,26,1,0,4
2025-02-28 00:00:00,84822.58,84159.05,84657.34,984.01,8.314589e+07,84200.67,1,85264.9565,41.352537,-461.131373,-399.498200,84657.34,84636.47,27,1,0,0
2025-02-28 01:00:00,84330.96,81131.57,84200.67,6654.60,5.480585e+08,81614.91,1,85054.5130,27.856612,-667.819526,-453.162465,84200.67,84657.34,27,1,0,0
2025-02-28 02:00:00,82815.88,79532.20,81614.91,9936.50,8.026110e+08,80759.62,1,84782.1905,24.955427,-890.372472,-540.604467,81614.91,84200.67,27,1,0,0
2025-02-28 03:00:00,81495.86,80192.01,80759.62,4598.99,3.718406e+08,80375.99,1,84489.6555,23.760120,-1085.193584,-649.522290,80759.62,81614.91,27,1,0,0


#### Handle Continuous Data

In [24]:
df.columns

Index(['high', 'low', 'open', 'volumefrom', 'volumeto', 'close',
       'conversionType', 'MA20', 'RSI', 'MACD', 'MACD_signal', 'Lag_1',
       'Lag_2', 'day', 'year', 'month', 'day_name'],
      dtype='object')

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
scaler = StandardScaler()

In [27]:
df[['high', 'low', 'open', 'volumefrom', 'volumeto',
        'MA20', 'RSI', 'MACD', 'MACD_signal']] = scaler.fit_transform(df[['high', 'low', 'open', 'volumefrom', 'volumeto',
        'MA20', 'RSI', 'MACD', 'MACD_signal']])

In [28]:
df.head(2)

,high,low,open,volumefrom,volumeto,close,conversionType,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2,day,year,month,day_name
time,,,,,,,,,,,,,,,,,
2025-02-27 23:00:00,-0.616647,-0.596678,-0.600739,0.041522,0.009376,84657.34,1,-0.510539,-0.589443,-1.008579,-0.940813,84636.47,84299.28,26,1,0,4
2025-02-28 00:00:00,-0.621269,-0.619614,-0.598045,-0.052970,-0.084771,84200.67,1,-0.516342,-0.897068,-1.022514,-0.972827,84657.34,84636.47,27,1,0,0


#### Define target and predictors

In [29]:
import numpy as np
df['day_sin'] = np.sin(2 * np.pi * df['day_name'] / 7)
df['day_cos'] = np.cos(2 * np.pi * df['day_name'] / 7)

In [31]:
x = df.drop(['close', 'day_name'], axis = 1)
y = df['close'].shift(-1)   #This is to make sure that this hours data is predicted by the info from the past hourx

In [32]:
x.shape, y.shape   #After the shift, the both data remains the same length as the last instance would be of type 'Nan'

((1968, 17), (1968,))

#### Split data into train and test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, shuffle=False, random_state = 69, train_size = 0.8)

## Build Model

In [ ]:
#import model and dependencies
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
#intialize model and define parameters
modelRF = RandomForestRegressor(n_estimators=100,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=5,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=30,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,)

#### Fit model

In [ ]:
len(x)

In [ ]:
modelRF.fit(x_train,y_train)

#### Evaluate model

In [ ]:
modelRF.score(x_train,y_train)

In [ ]:
print("model performance on train", modelRF.score(x_train, y_train),
      f"\nmodel performance on unseen data",  modelRF.score(x_test[:393], y_test[:393]))

#### Use a simpler model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
modelLR = LinearRegression(fit_intercept=True,
    n_jobs=50,
    positive=False,)

In [ ]:
modelLR.fit(x_train, y_train)

In [ ]:
len(x_test)

In [ ]:
print("model performance on train", modelLR.score(x_train, y_train),
      f"\nmodel performance on unseen data",  modelLR.score(x_test[:393], y_test[:393]))

In [ ]:
modelLR.predict(x_test[391:392])

In [ ]:
y_test[391:392]